In [67]:
import pandas as pd
import pronouncing as prn
import string
import re
import torch

In [73]:
# Takes in a text file with all the poems returns a list of poems with titles
import itertools
def text_to_poems():
   
    with open('sonnets.txt', 'r') as file:
        poems_list = file.read().split('\n\n')
        
        # Poems and their titles got split up when parsing - reunite them
        poems_clean = []
        for i in range(0, (int(len(poems_list) - 1)), 2):
            poem_with_title = poems_list[i] + " " + poems_list[i+1]
            poems_clean.append(poem_with_title)
            
        # Turn each poem to a list of lists of sentences of lowercase words, no punctuation
        poems = []
        for poem in poems_clean:
            poem_phonemes = []
            #poem = poem.lower()
            poem_no_punc = poem.translate(str.maketrans('','',string.punctuation))
            poems.append(poem_no_punc)#.split('\n')  
               
    return poems

text_to_poems()

['I FROM fairest creatures we desire increase\nThat thereby beautys rose might never die\nBut as the riper should by time decease\nHis tender heir might bear his memory\nBut thou contracted to thine own bright eyes\nFeedst thy lightst flame with selfsubstantial fuel\nMaking a famine where abundance lies\nThyself thy foe to thy sweet self too cruel\nThou that art now the worlds fresh ornament\nAnd only herald to the gaudy spring\nWithin thine own bud buriest thy content\nAnd tender churl makest waste in niggarding\nPity the world or else this glutton be\nTo eat the worlds due by the grave and thee',
 'II When forty winters shall beseige thy brow\nAnd dig deep trenches in thy beautys field\nThy youths proud livery so gazed on now\nWill be a tatterd weed of small worth held\nThen being askd where all thy beauty lies\nWhere all the treasure of thy lusty days\nTo say within thine own deepsunken eyes\nWere an alleating shame and thriftless praise\nHow much more praise deserved thy beautys us

In [72]:
# Make all words lowercase and make each line into a list of word. Returns list of lists
def poems_to_list(poems):
    poems_list = []
    for poem in poems:
        lines = list(map(lambda x:x.lower(), poem.split('\n')))
        poem= [l.split() for l in lines]
        poems_list.append(poem)
        
    return poems_list

#poems= text_to_poems()
#poems_words = poems_to_list(poems)

In [68]:
def words_to_phonemes(poems):
    phonemes = []
    slang = []
    for poem in poems:
        for line in poem:
            #print(line)
            for word in line:
                #print(word)
                p = prn.phones_for_word(word)
    
                if len(p) == 1:
                    phonemes.append(p)
                elif len(p) > 1:
                    # TODO fix this hack by figuring out which pronunciation is best instead of just choosing the 1st
                    phonemes.append([p[0]])
                else:
                    slang.append(word)
                    phonemes.append(['S'])
        
        
    return phonemes

poems_clean= text_to_poems()
poems = poems_to_list(poems_clean)
phonemes = words_to_phonemes(poems)

In [84]:
def clean_poem(poem):
    words = []
    poem_phonemes = []
    poem = poem.lower()
    poem_no_punc = poem.translate(str.maketrans('','',string.punctuation))
    words.append(poem_no_punc)#.split('\n'))
    words = [word for word in words]
    for word in words:
        poem_phonemes.append(prn.phones_for_word(word))
                             
    return poem_phonemes


a = map(clean_poem, [poem for poem in poems])


In [ ]:

        for word in words:
            poem_phonemes.append(prn.phones_for_word(word))

In [446]:
# TODO when there are multiple pronunciations available, choose the one that fits that 
# metrical pattern
# calculate the metrical pattern of the whole poem -> use this to find the local pattern
# use the local pattern to figure out what stress we want to put in
# compare stresses of different pronunciations, and choose the first one that matches
#

In [ ]:
# Get the sylable count, and use it to make the data into a matrix
# Put every 10 sylables into a list or matrix -> should I use numpy?


In [60]:
  #data = file.read().split('\n\n')
        #print(poems_list)
        #poems_list = [item.lower() for item.split('\n') in data]

        #poems_list = [p.replace('\n',' ') for p in data]
                
        #print(poems_clean[1])
        #print(poem.split('\n')[0:10] for poem in poems_clean)        
        #print(type(poems_list[100].split('\n')[0]))
        #print(poems_list[101].split('\n')[0:100])
                      